In [2]:
from azureml.core import Workspace, Experiment

In [3]:
#ws = Workspace.get(name="udacity-project")
ws = Workspace.from_config()

In [4]:
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: azmldatad01
Azure region: eastus2
Subscription id: 8723c91c-b9c0-4b54-b0e0-8561d583e19b
Resource group: rsgdatad01


In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

cluster_name = "cpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config) ## creating the cluster

compute_target.wait_for_completion(show_output=True)

Found existing compute target
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
import shutil

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.01, 0.1, 0.5, 1.0, 1.5, 2.0, 3.0),
    "--max_iter": choice(50, 100, 150, 200)
}
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

script_path = "./training"
script_name = 'train.py'
if "training" not in os.listdir():
    os.mkdir(script_path)
    shutil.copy(script_name, script_path)

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=script_path, entry_script=script_name, compute_target=compute_target)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=24,
                                    max_concurrent_runs=4
                                    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)

In [8]:
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_85eb6fad-86c7-4cd0-8a0d-70e35e411dfe,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_details = best_run.get_details()

print("Best Run ID: {0}".format(best_run.id))
print("Accuracy: {0:.3f}".format(best_run_metrics["Accuracy"]))
print("Parameters: {0}".format(best_run_details["runDefinition"]["arguments"]))

joblib.dump(value=best_run, filename="outputs/best_model.joblib")
best_model = best_run.register_model(model_name="best_model", model_path="outputs/best_model.joblib")



In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=ds_url)
df = ds.to_pandas_dataframe()
df.head(10)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=66)

df_train = pd.concat([X_train, y_train], axis=1).reset_index(drop=True)
df_train.to_csv("bank_train_data.csv", index=False)
df_train.head()

In [ ]:
if not os.path.isdir("dataset"):
    os.mkdir("dataset")
    
data_store = ws.get_default_datastore()
data_store.upload(src_dir="./dataset", target_path="bankmarketing", overwrite=True, show_progress=True)

# Uploading the training data as a tabular dataset
ds_train = TabularDatasetFactory.from_delimited_files(path=data_store.path("bankmarketing/bank_train_data.csv"))

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    compute_target=compute_target,
    training_data=df_train,
    label_column_name="y",
    n_cross_validations=5)

In [ ]:
# Submit your automl run

exp_2 = Experiment(ws, "automl-experiment")
run_2 = exp_2.submit(automl_config, show_output=False)

In [ ]:
exp_2

In [ ]:
run_2

In [ ]:
# Retrieve and save your best automl model.

best_run, model = local_run.get_output()
joblib.dump(model, "outputs/best_automl_model.joblib")
best_run.register_model("auto-ml-model" , model_path = "./outputs/")